# Parallel Rabi Experiment with Interactive Analysis

This notebook demonstrates how to run Rabi oscillation experiments in parallel on OQTOPUS Qulacs backend and analyze the results using pandas DataFrames and interactive plotly visualizations.

## Key Features
- **Parallel execution** for faster experiments
- **Pandas DataFrame** results for easy data analysis
- **Interactive plotly** visualizations
- **Automatic transpilation** to physical qubits
- **Parameter embedding** for reproducible results

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from oqtopus_experiments.backends import OqtopusBackend
from oqtopus_experiments.experiments import Rabi

## 1. Setup Backend and Experiment

We'll use the OQTOPUS Qulacs backend for noiseless quantum simulation.

In [2]:
# Initialize OQTOPUS backend for Qulacs
backend = OqtopusBackend(device="qulacs")
print(f"Backend initialized: {backend.device_name}")
print(f"Backend available: {backend.available}")

✅ OQTOPUS backend initialized (device: qulacs, timeout: 120s)
Backend initialized: qulacs
Backend available: True


In [3]:
# Create Rabi experiment with physical qubit specification
rabi = Rabi(
    experiment_name="parallel_rabi_notebook",
    physical_qubit=3,  # Target physical qubit for transpilation
    amplitude_points=15,  # More points for smoother curve
    max_amplitude=2.0,  # Cover full 2π range
)

print(f"Experiment: {rabi.experiment_name}")
print(f"Physical qubit: {rabi.physical_qubit}")
print(f"Amplitude points: {rabi.amplitude_points}")
print(f"Max amplitude: {rabi.max_amplitude}")

Results: .results/parallel_rabi_notebook_20250720_142515
Rabi: parallel_rabi_notebook
Experiment: parallel_rabi_notebook
Physical qubit: 3
Amplitude points: 15
Max amplitude: 2.0


## 2. Generate Quantum Circuits

The circuits will be automatically transpiled to the specified physical qubit.

In [4]:
# Generate circuits with automatic transpilation
circuits = rabi.circuits()
print(f"Generated {len(circuits)} circuits")
print(f"Circuit collection type: {type(circuits)}")

Created 15 Rabi circuits (amplitude range: 0.000 - 2.000)
Physical qubit 3 specified, performing transpilation...
✅ Device 'anemone' information loaded successfully
✅ Transpiled 15 circuits to physical qubit 3
Generated 15 circuits
Circuit collection type: <class 'oqtopus_experiments.models.circuit_collection.CircuitCollection'>


In [5]:
# Examine a sample transpiled circuit
print("Sample transpiled circuit (amplitude ≈ 0.5):")
print(circuits[6].draw())

Sample transpiled circuit (amplitude ≈ 0.5):
                                                                    
  ancilla_0 -> 0 ───────────────────────────────────────────────────
                                                                    
  ancilla_1 -> 1 ───────────────────────────────────────────────────
                                                                    
  ancilla_2 -> 2 ───────────────────────────────────────────────────
                 ┌─────────┐┌────┐┌───────────┐┌────┐┌──────────┐┌─┐
        q_0 -> 3 ┤ Rz(π/2) ├┤ √X ├┤ Rz(13π/7) ├┤ √X ├┤ Rz(5π/2) ├┤M├
                 └─────────┘└────┘└───────────┘└────┘└──────────┘└╥┘
  ancilla_3 -> 4 ─────────────────────────────────────────────────╫─
                                                                  ║ 
  ancilla_4 -> 5 ─────────────────────────────────────────────────╫─
                                                                  ║ 
  ancilla_5 -> 6 ─────────────────────────────────────────

## 3. Parallel Execution

Run all circuits in parallel for efficient data collection.

In [6]:
# Execute experiments in parallel
result = rabi.run_parallel(
    backend=backend, 
    shots=1000,  # Number of measurements per circuit
    workers=4    # Parallel workers
)

print(f"Experiment completed successfully!")
print(f"Result type: {type(result)}")

Running Rabi in parallel
Using pre-created circuits: 15 circuits
Auto-transpiling circuits: logical qubit 0 → physical qubit 3
❌ Failed to load device 'qulacs': 'rzx90'
✅ Transpilation successful
Transpiled circuit sample:
                                                                   
  ancilla_0 -> 0 ──────────────────────────────────────────────────
                                                                   
  ancilla_1 -> 1 ──────────────────────────────────────────────────
                                                                   
  ancilla_2 -> 2 ──────────────────────────────────────────────────
                 ┌─────────┐┌────┐┌──────────┐┌────┐┌──────────┐┌─┐
        q_0 -> 3 ┤ Rz(π/2) ├┤ √X ├┤ Rz(8π/7) ├┤ √X ├┤ Rz(5π/2) ├┤M├
                 └─────────┘└────┘└──────────┘└────┘└──────────┘└╥┘
  ancilla_3 -> 4 ────────────────────────────────────────────────╫─
                                                                 ║ 
  ancilla_4 -> 5 ────────────

## 4. Data Analysis with Pandas DataFrame

Analyze results using pandas for powerful data manipulation.

In [ ]:
# Convert results to pandas DataFrame (always returns DataFrame now)
df = result.analyze(plot=True, save_data=False)
print(f"DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()

In [8]:
# Display fitting parameters
fitting_params = df[['device', 'pi_amplitude', 'frequency', 'r_squared']].drop_duplicates()
print("Fitting Results:")
print(fitting_params)

Fitting Results:
    device  pi_amplitude  frequency  r_squared
0  backend      0.459118   1.089044   0.045935


In [9]:
# Statistical summary
print("Data Summary:")
print(df[['amplitude', 'probability']].describe())

Data Summary:
       amplitude  probability
count  15.000000    15.000000
mean    1.000000     0.466200
std     0.638877     0.377532
min     0.000000     0.000000
25%     0.500000     0.117117
50%     1.000000     0.392392
75%     1.500000     0.810811
max     2.000000     1.000000


## 5. Interactive Visualization

Create an interactive plotly visualization showing the Rabi oscillation with fitted curve.

In [ ]:
# Generate interactive plot (DataFrame is returned automatically)
df_with_plot = result.analyze(plot=True, save_data=False)
print("Interactive plot created! Check the output above.")

## 6. Advanced Analysis

Perform additional analysis using pandas functionality.

In [ ]:
# Find π-pulse amplitude
pi_amplitude = df['pi_amplitude'].iloc[0]
frequency = df['frequency'].iloc[0]
r_squared = df['r_squared'].iloc[0]

print(f"Analysis Results:")
print(f"   π-pulse amplitude: {pi_amplitude:.4f}")
print(f"   Rabi frequency: {frequency:.4f}")
print(f"   Fit quality (R²): {r_squared:.4f}")

# Theoretical π-pulse positions
pi_positions = [pi_amplitude * (2*n + 1) for n in range(3)]
print(f"\nTheoretical π-pulse positions: {[f'{p:.3f}' for p in pi_positions]}")

In [ ]:
# Find data points near π-pulses
tolerance = 0.1
pi_data = []

for i, pi_pos in enumerate(pi_positions):
    near_pi = df[abs(df['amplitude'] - pi_pos) < tolerance]
    if not near_pi.empty:
        closest = near_pi.iloc[0]
        pi_data.append({
            'π_pulse': f'π×{2*i+1}',
            'amplitude': closest['amplitude'],
            'probability': closest['probability']
        })

pi_df = pd.DataFrame(pi_data)
print("\nData points near π-pulses:")
print(pi_df)

In [ ]:
# Export results to CSV for further analysis
csv_filename = f"rabi_results_{rabi.experiment_name}.csv"
df.to_csv(csv_filename, index=False)
print(f"\nResults saved to: {csv_filename}")

# Show file info
import os
file_size = os.path.getsize(csv_filename)
print(f"   File size: {file_size} bytes")
print(f"   Rows exported: {len(df)}")

## 7. Summary

This notebook demonstrated:

- **Parallel Rabi experiment** execution on OQTOPUS Qulacs  
- **Automatic transpilation** to physical qubits  
- **DataFrame conversion** for easy data analysis  
- **Interactive plotly visualization** with fitting curves  
- **Advanced pandas analysis** for detailed insights  
- **Data export** for reproducible research  

The combination of parallel execution, DataFrame results, and interactive visualization makes quantum experiment analysis both efficient and insightful.

In [ ]:
# Final summary
print("Parallel Rabi Analysis Complete!")
print(f"   Experiment: {rabi.experiment_name}")
print(f"   Physical qubit: {rabi.physical_qubit}")
print(f"   Data points: {len(df)}")
print(f"   π-pulse amplitude: {pi_amplitude:.4f}")
print(f"   Fit quality: {r_squared:.4f}")
print(f"   Results saved: {csv_filename}")

In [16]:
from plotly import graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
	x=df['amplitude'],
	y=df['probability'],
	mode='markers',
	marker=dict(size=8, color='blue', opacity=0.6),
	name='Data Points'
))

fig.update_layout(
	title='Rabi Experiment Results',
	xaxis_title='Amplitude',
	yaxis_title='Probability',
	legend=dict(x=0.01, y=0.99),
	template='plotly_white'
)
fig.show()	

In [26]:
x = df['amplitude']
y = df['probability']
COLORS = [
    "#0C5DA5",
    "#00B945",
    "#FF9500",
    "#FF2C00",
    "#845B97",
    "#474747",
    "#9e9e9e",
]

fig = go.Figure()
fig.add_trace(
	go.Scatter(
	x=x,
	y=y,
	line=dict(color=COLORS[1]),
	mode="scatter",
	name="Fit",
	)
)


ValueError: 
    Invalid value of type 'builtins.str' received for the 'mode' property of scatter
        Received value: 'scatter'

    The 'mode' property is a flaglist and may be specified
    as a string containing:
      - Any combination of ['lines', 'markers', 'text'] joined with '+' characters
        (e.g. 'lines+markers')
        OR exactly one of ['none'] (e.g. 'none')